In [ ]:
# Assignment 2 for Group44

In [12]:
"""
In a warehouse, the robot is used for material handling. Your task is to help the robot to place the 
item on one of the available shelves. Here, the warehouse layout is represented by an m×n grid of 
characters, where the grid includes "R", "#", "@", "$", "T", and :
• "R" stands for the robot. 
• "$" denotes the item that needs to be moved. There’s only one item on the grid.
• "#" denotes an open space, which both the robot and the item can move onto.
• "@" denotes an obstacle. Neither the robot nor the item can move onto it.
• "T" denotes an empty shelf where the item can be placed. If there are multiple empty shelves, the robot can 
place the item on any of them. Besides, the robot can move through the empty shelf "T".
Available actions of the robot are as follows:
• “Move”: The robot can move up, down, left or right on the grid to reach adjacent open spaces ("#").
• “Push”: The robot can push the item by standing next to it and moving in the direction of the item. The robot 
cannot move through the item.
You should return the minimum number of pushes required for the robot to move the item to an 
empty shelf. If it's impossible to place the item on any shelf, return -1.
• Hint 1: Consider calculating the minimum pushes required to reach different target shelves separately.
• Hint 2: Consider utilizing A* search and heuristically use the Manhattan distance between the item and the 
target shelf to guide the search.
"""
from typing import List
import heapq


def solve(grid: List[List[str]]) -> int:
    """Your solution to the problem goes in this function.
    :param:
        grid (List[List[str]]): The warehouse layout, e.g., [["@", "@", "@"], ["@", "R", "$"], ["@", "@", "T"]]
    :return:
        int: the minimum number of pushes required for the robot to move the item to an empty shelf.
        return -1 if no solution
        return -2 if invalid input
        return the minimum number of pushes required for the robot to move the item to an empty shelf
    """
    # if the input is not 2d array of strings, return -2
    if not isinstance(grid, list) or not all(isinstance(row, list) for row in grid) or not all(
            isinstance(item, str) for row in grid for item in row):
        print('Invalid input: not a 2d array of strings')
        return -2
    # if the input is not a valid grid, the length is not equal to width, return -2
    if not all(len(row) == len(grid[0]) for row in grid):
        print('Invalid input: not a valid grid, length is not equal to width')
        return -2
    # if the input is not a valid grid, contain charactors other than @#$RT return -2
    if not all(item in ['@', '#', '$', 'R', 'T'] for row in grid for item in row):
        print('Invalid input: not a valid grid, contain charactors other than @#$RT')
        return -2
    # if the input is not a valid grid, contain more than one robot return -2
    if sum(row.count('R') for row in grid) != 1:
        print('Invalid input: not a valid grid, contain more than one robot')
        return -2
    # if there are more than on item, return -2
    if sum(row.count('$') for row in grid) != 1:
        print('Invalid input: not a valid grid, contain more than one item')
        return -2
    # if there are no target shelf, return -2
    if sum(row.count('T') for row in grid) == 0:
        print('Invalid input: not a valid grid, contain no target shelf')
        return -2

    # Find the robot, item, and shelves
    robot = item = shelves = None
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'R':
                robot = (i, j)
            elif grid[i][j] == '$':
                item = (i, j)
            elif grid[i][j] == 'T':
                if shelves is None:
                    shelves = []
                shelves.append((i, j))

    # if the x and y of the item is both blocked, return -1
    if block_x(grid, item[0], item[1]) and block_y(grid, item[0], item[1]):
        print('Invalid input: the item is blocked')
        return -1

    # if the distance between the item and the target shelf is 1, return 1
    if any(heuristic(item, shelf) == 1 for shelf in shelves):
        return 1
    
    min_pushes = float('inf')
    for shelf in shelves:
        _, cost_robot_to_item = astar(grid, robot, item)
        _, cost_item_to_shelf = astar(grid, item, shelf)
        pushes = cost_robot_to_item.get(item, float('inf')) + cost_item_to_shelf.get(shelf, float('inf'))
        min_pushes = min(min_pushes, pushes)

    return min_pushes if min_pushes != float('inf') else -1


def block_x(grid, x, y):
    if grid[x - 1][y] == '@' or grid[x + 1][y] == '@' or x + 1 >= len(grid) or x - 1 < 0:
        return True
    return False


def block_y(grid, x, y):
    if grid[x][y - 1] == '@' or grid[x][y + 1] == '@' or y + 1 >= len(grid[0]) or y - 1 < 0:
        return True
    return False


def heuristic(a, b):
    return abs(b[0] - a[0]) + abs(b[1] - a[1])


def astar(grid, start, goal):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    while frontier:
        _, current = heapq.heappop(frontier)

        if current == goal:
            break

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            next = (current[0] + dx, current[1] + dy)
            if 0 <= next[0] < len(grid) and 0 <= next[1] < len(grid[0]) and grid[next[0]][next[1]] != '@':
                new_cost = cost_so_far[current] + 1
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + heuristic(goal, next)
                    heapq.heappush(frontier, (priority, next))
                    came_from[next] = current

    return came_from, cost_so_far


In [10]:
# test case 1
grid1 = [["@", "@", "@", "@", "@", "@"],
         ["@", "@", "@", "@", "T", "@"],
         ["@", "#", "$", "#", "#", "@"],
         ["@", "#", "@", "@", "#", "@"],
         ["@", "R", "#", "#", "#", "@"],
         ["@", "T", "@", "@", "@", "@"]]
answer1 = 3
result1 = solve(grid1)
assert result1 == answer1, f"Test case 1: expected {answer1}, got {result1}"
print('Passed test case 1...')

AssertionError: Test case 1: expected 3, got 6

In [11]:
# test case 2
grid2 = [["@", "T", "@", "@", "@", "@"],
         ["@", "#", "@", "@", "@", "@"],
         ["@", "#", "#", "#", "$", "@"],
         ["@", "#", "@", "@", "@", "@"],
         ["@", "R", "#", "#", "T", "@"],
         ["@", "@", "@", "@", "@", "@"]]
answer2 = -1
result2 = solve(grid2)
assert result2 == answer2, f"Test case 2: expected {answer2}, got {result2}"
print('Passed test case 2...')

Invalid input: the item is blocked
Passed test case 2...
